<a href="https://colab.research.google.com/github/jamagiwa/Kaggle/blob/main/R_bulk_RNAseq_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RでbulkRNAseqの解析練習を行う



パッケージのダウンロード

In [1]:
install.packages("BiocManager")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
BiocManager::install(c("edgeR", "airway", "pheatmap", "ggplot2", "dplyrr"))

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'ggplot2'”
Installing package(s) 'BiocVersion', 'edgeR', 'airway', 'pheatmap', 'dplyrr'

Warning message:
“package ‘dplyrr’ is not available for Bioconductor version '3.21'

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
also installing the dependencies ‘matrixStats’, ‘XVector’, ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘abind’, ‘SparseArray’, ‘statmod’, ‘MatrixGenerics’, ‘GenomicRanges’, ‘Biobase’, ‘BiocGenerics’, ‘S4Vectors’, ‘IRanges’, ‘GenomeInfoDb’, ‘S4Arrays’, ‘DelayedArray’

In [10]:
BiocManager::install("airway")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Installing package(s) 'airway'

also installing the dependencies ‘SparseArray’, ‘DelayedArray’, ‘SummarizedExperiment’


Old packages: 'devtools', 'systemfonts'



In [11]:
library(edgeR)
library(pheatmap)
library(ggplot2)
library(dplyr)
library(airway)

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, 

In [17]:
data(airway)
#airwayはSummarizedExperiment形式
sample <- airway

In [18]:
head(sample)

class: RangedSummarizedExperiment 
dim: 6 8 
metadata(1): ''
assays(1): counts
rownames(6): ENSG00000000003 ENSG00000000005 ... ENSG00000000460
  ENSG00000000938
rowData names(10): gene_id gene_name ... seq_coord_system symbol
colnames(8): SRR1039508 SRR1039509 ... SRR1039520 SRR1039521
colData names(9): SampleName cell ... Sample BioSample

#データの整理

In [19]:
#カウントデータの行列
counts_matrix <- assay(sample)
print(counts_matrix)

                SRR1039508 SRR1039509 SRR1039512 SRR1039513 SRR1039516
ENSG00000000003        679        448        873        408       1138
ENSG00000000005          0          0          0          0          0
ENSG00000000419        467        515        621        365        587
ENSG00000000457        260        211        263        164        245
ENSG00000000460         60         55         40         35         78
ENSG00000000938          0          0          2          0          1
ENSG00000000971       3251       3679       6177       4252       6721
ENSG00000001036       1433       1062       1733        881       1424
ENSG00000001084        519        380        595        493        820
ENSG00000001167        394        236        464        175        658
ENSG00000001460        172        168        264        118        241
ENSG00000001461       2112       1867       5137       2657       2735
ENSG00000001497        524        488        638        357        676
ENSG00

In [21]:
#メタデータ
meta_info <- as.data.frame(colData(sample))
print(meta_info)

           SampleName    cell   dex albut        Run avgLength Experiment
SRR1039508 GSM1275862  N61311 untrt untrt SRR1039508       126  SRX384345
SRR1039509 GSM1275863  N61311   trt untrt SRR1039509       126  SRX384346
SRR1039512 GSM1275866 N052611 untrt untrt SRR1039512       126  SRX384349
SRR1039513 GSM1275867 N052611   trt untrt SRR1039513        87  SRX384350
SRR1039516 GSM1275870 N080611 untrt untrt SRR1039516       120  SRX384353
SRR1039517 GSM1275871 N080611   trt untrt SRR1039517       126  SRX384354
SRR1039520 GSM1275874 N061011 untrt untrt SRR1039520       101  SRX384357
SRR1039521 GSM1275875 N061011   trt untrt SRR1039521        98  SRX384358
              Sample    BioSample
SRR1039508 SRS508568 SAMN02422669
SRR1039509 SRS508567 SAMN02422675
SRR1039512 SRS508571 SAMN02422678
SRR1039513 SRS508572 SAMN02422670
SRR1039516 SRS508575 SAMN02422682
SRR1039517 SRS508576 SAMN02422673
SRR1039520 SRS508579 SAMN02422683
SRR1039521 SRS508580 SAMN02422677


In [22]:
#遺伝子情報
gene_info <- as.data.frame(rowRanges(sample))
print(gene_info)

      group      group_name seqnames     start       end width strand exon_id
1         1 ENSG00000000003        X  99883667  99884983  1317      -  667145
2         1 ENSG00000000003        X  99885756  99885863   108      -  667146
3         1 ENSG00000000003        X  99887482  99887565    84      -  667147
4         1 ENSG00000000003        X  99887538  99887565    28      -  667148
5         1 ENSG00000000003        X  99888402  99888536   135      -  667149
6         1 ENSG00000000003        X  99888402  99888536   135      -  667150
7         1 ENSG00000000003        X  99888439  99888536    98      -  667151
8         1 ENSG00000000003        X  99888928  99889026    99      -  667153
9         1 ENSG00000000003        X  99888928  99889026    99      -  667152
10        1 ENSG00000000003        X  99890175  99890249    75      -  667154
11        1 ENSG00000000003        X  99890175  99890249    75      -  667155
12        1 ENSG00000000003        X  99890555  99890743   189  

DEGListにデータを格納

In [28]:
#比較する実験群をgroup
dge <- DGEList(counts = counts_matrix, group = meta_info$dex)

#フィルタリング
#低発現遺伝子の除去
keep <- rowSums(cpm(dge) > 1) >= 4
dge <- dge[keep, , keep.lib.sizes = FALSE]

#TMM正規化
#総リード数の補正
dge <- calcNormFactors(dge)
dge

,SRR1039508,SRR1039509,SRR1039512,SRR1039513,SRR1039516,SRR1039517,SRR1039520,SRR1039521
ENSG00000000003,679,448,873,408,1138,1047,770,572
ENSG00000000419,467,515,621,365,587,799,417,508
ENSG00000000457,260,211,263,164,245,331,233,229
ENSG00000000460,60,55,40,35,78,63,76,60
ENSG00000000971,3251,3679,6177,4252,6721,11027,5176,7995
ENSG00000001036,1433,1062,1733,881,1424,1439,1359,1109
ENSG00000001084,519,380,595,493,820,714,696,704
ENSG00000001167,394,236,464,175,658,584,360,269
ENSG00000001460,172,168,264,118,241,210,155,177
ENSG00000001461,2112,1867,5137,2657,2735,2751,2467,2905
